# An Introduction to **Asynchronous/Concurrent** Programming in Kotlin

- A brief description of `runBlocking` and `coroutineScope` is given.
- The concept of `suspend` functions is explained.
- `launch`ing a coroutine is covered.

**HINT**: You may need to read this section (and [Async/Concurrent: Basics](12-async-concurrent-basics.ipynb), [Async/Concurrent: Coroutine Context](13-async-concurrent-context.ipynb)) **multiple times** and revisit it for review and practice, as it can be **complex** for all to fully grasp on the first pass. Don't worry! This isn't a reflection of your ability; the concept itself can be quite **confusing** and often **requires careful consideration** and **experimentation** to fully understand.

In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors

## Introducing **suspend functions**, **coroutines**, `runBlocking` and `launch`

- `runBlocking` is a **coroutine builder** with a given or inherited **context** that creates a **scope** for us `{...}`.
- `coroutineScope` is a scope `{...}` with a given **context** where your **coroutines/codes** executed in.
- **Coroutines** and **suspend functions** can be **launched/executed** in the `runBlocking {...}` scope `{...}` (or other coroutine scopes).
- **Suspend functions** can be directly called or **launched** in a coroutine **scope** `{...}`.
- `runBlocking` **blocks** the given thread for its own usage.
- There are more concepts like `Dispatcher`, `job`, `contex`, ... that will be dicusses later.

In [2]:
// Creating the entry point using runBlocking

/* 
'runBlocking' is a coroutine builder that bridges your program entry 
to an async/concurrent programming structure (coroutines).
*/

// Kotlin concurrent programs normally begin with a runBlocking {} bridge.
// runBlocking, as its name suggests, blocks its thread in which the code is being executed.
// runBlocking is a coroutine builder and defines a coroutine scope in '{...}' so other coroutines can be executed in it.
fun main() = runBlocking { // A coroutine scope is defined here
    //.............runBlocking coroutine scope.................
    println("Hello World")
}
main()

Hello World


In [3]:
// Creating a simple suspend function

// The keyword 'suspend' is added to specify that the function might get suspended.
suspend fun fetch() { 
    // There is no suspension point here in this function yet.
    println("Hello World")
}

## The order of code execution in a **coroutine scope**

1. Code execution continues with normal (non-suspend and non-coroutine) codes.
   * `launch`ing a **job** won't **suspend** execution; just adds coroutines to the execution queue and skips the launched coroutine. 
3. If the code reaches at a **suspension point** and a **suspend** function pauses the code execution, it starts running all **already launched jobs** unitl *they complete or suspend again*.
4. Then code execution *continues*; if there is no more suspending point at the end of the scope, all queued jobs that already have launched get executed *unitl they are all done*.
5. Finally, coroutine scope finishes running *when verything is DONE* and then block returns.

In [4]:
// Manually suspending the function

// Adding 'delay' to suspend the function
suspend fun fetch() {  // This is a suspendable function
    println("'fetch' started; wait for 3 sec ...")
    
    // Delay is a 'suspending function' that suspends 'fetch'
    delay(3_000)  // 1_000 = 1s
    
    println("Hello World")
}

// This code runs as a regular code
fun main() = runBlocking {
    println("Program started.")
    // 'fetch' is executed in runBocking scope
    fetch() // Program pauses (suspends) here, because coroutine scopes pause when they reach a suspendable function, then continues.
    println("Program finished.")
}
main()

Program started.
'fetch' started; wait for 3 sec ...
Hello World
Program finished.


In [5]:
// Using 'launch' to postpone 'suspend' function execution as a 'job'

suspend fun fetch() {  
    println("'fetch' started; wait for 3 sec ...")
    delay(3_000)  // 1_000 = 1s
    println("Hello World")
}

// Now, using 'launch' coroutine builder, the suspend function is launched as a job in runBlocking coroutine scope.
fun main() = runBlocking { 
    println("Program started.")
    
    // 'launch' is another coroutine builder that launches a coroutine and returns a 'job' in a given 'coroutineScope'.
    /* 
    The coroutine 'job' will get executes:
        - when all non-suspendable (normal) codes in the runBlocking block finish and there is nothing left in its scope.
        - or when runBlocking gets suspended at some point. 
            In this case, all already launched jobs will start executing until they finish or suspend again.
    */
    // runBlocking will not pause here.
    launch { // This launches a coroutine, returns a 'job' and postpones its execution (we have ignored the returned 'job' here).
        fetch() 
    }
    println("Program finished.")
}
main()

Program started.
Program finished.
'fetch' started; wait for 3 sec ...
Hello World


In [6]:
// Adding another suspending point in runBlocking scope

suspend fun fetch() {  
    println("'fetch' started; wait for 3 sec ...")
    delay(3_000)  // 1_000 = 1s
    println("Hello World")
}

fun main() = runBlocking { 
    println("Program started.")

    // REMEMBER: 'launch' can only launch coroutines in a coroutine scope
    launch { // This launches a coroutine and postpones its execution.
        fetch() 
    }
    
    // runBlocking pauses here and the launched job will start running
    delay(5_000) // Change 5000 to 1000 to see how execution order changes
    println("Program finished.")
}
main()

Program started.
'fetch' started; wait for 3 sec ...
Hello World
Program finished.


## Experimenting if `runBlocking` actally blocks its thread

You can read a good [article about it here by Jim Steinberger](https://www.baeldung.com/kotlin/coroutines-runblocking-coroutinescope). I have used some ideas of this valuable resource.

- **IMPORTANT NOTE:** Never block your threads unless you know what you are doing. `runBlocking` is designed for main program entry and bridging between normal and async/concurrent codes. This example is given to better illustrate how it works.


In [7]:
// Blocking thread

val threadNo = 1 // Change to 2 threads to see what happens; you can see concurrent execution

// Creating a custom coroutine context with threadNo thread
val context = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()

/* 
'runBlocking' is a coroutine builder that inherits 
its 'context' from outer scope (the scope its is launched) unless a 'context' is given.
The coroutine builder 'runBlocking {...}' does not return a suspend function.
*/
fun runBlockingCorScope() = runBlocking {
    println(Thread.currentThread().toString()+" runBlockingCorScope launch; wait...")
    delay(10_000)
    println(Thread.currentThread().toString()+" runBlockingCorScope finished")
}

fun main() = runBlocking(context) { // the 'context' is given to the root 'runBlocking'
    println("Program started.")

    // Launching jobs to check when thread gets block
    launch {
        for (k in 1..20) {
            println(Thread.currentThread().toString()+" I'm not blocked $k")
            delay(200)
        }
    }

    launch { runBlockingCorScope() }
    // runBlockingCorScope()  // Comment above code and uncomment this to see the difference

    println("Program finihsed.")
}
main()

context.close()  // Gracefully close the coroutine dispatcher

Program started.
Program finihsed.
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 1
Thread[#54,pool-1-thread-1,5,main] runBlockingCorScope launch; wait...
Thread[#54,pool-1-thread-1,5,main] runBlockingCorScope finished
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 2
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 3
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 4
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 5
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 6
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 7
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 8
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 9
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 10
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 11
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 12
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 13
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 14
Thread[#54,pool-1-thread-1,5,main] I'm not blocked 15
Thread[#54,pool-1-thread-1,5,